<a href="https://colab.research.google.com/github/bala1802/END_Assignments/blob/main/Session7_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
cd /content/drive/MyDrive/ENDProgram/Session7

/content/drive/MyDrive/ENDProgram/Session7


In [7]:
import pandas as pd

In [8]:
df_phrases = pd.read_csv('stanfordSentimentTreebank/dictionary.txt', sep='|', header=None)
df_labels = pd.read_csv('stanfordSentimentTreebank/sentiment_labels.txt', sep='|')

In [9]:
df = pd.merge(df_phrases, df_labels, how='inner', left_on=1, right_on='phrase ids')

In [10]:
def extractLabelFromScore(scoreValue):
  if scoreValue <= 0.2:
    return 0
  elif scoreValue <= 0.4:
    return 1
  elif scoreValue <= 0.6:
    return 2
  elif scoreValue <= 0.8:
    return 3
  else:
    return 4

In [11]:
df['label'] = df.apply(lambda row: extractLabelFromScore(row['sentiment values']), axis=1)

In [12]:
df.head()

,0,1,phrase ids,sentiment values,label
0,!,0,0,0.50000,2
1,! ',22935,22935,0.52778,2
2,! '',18235,18235,0.50000,2
3,! Alas,179257,179257,0.44444,2
4,! Brilliant,22936,22936,0.86111,4


In [13]:
df.shape

(239232, 5)

In [14]:
df.label.value_counts()

2    119449
3     50148
1     43028
4     15255
0     11352
Name: label, dtype: int64

In [15]:
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [22]:
fields = [('review', Review),('labels',Label)]

In [23]:
example = [data.Example.fromlist([df[0][i],df['label'][i]], fields) for i in range(df.shape[0])]

In [24]:
dataset = data.Dataset(example, fields)

In [25]:
(train, valid) = dataset.split(split_ratio=[0.85, 0.15])

In [26]:
(len(train), len(valid))

(203347, 35885)

In [27]:
vars(train.examples[13])

{'labels': 2,
 'review': ['Men',
  'in',
  'Black',
  'II',
  ',',
  "''",
  'has',
  'all',
  'the',
  'earmarks',
  'of',
  'a',
  'sequel',
  '.']}

In [28]:
Review.build_vocab(train)
Label.build_vocab(train)

In [ ]:
print('Size of input vocab : ', len(Review.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Review.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [30]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.review),
                                                            sort_within_batch=True, device = device)

In [31]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Review.vocab.stoi, tokens)

In [32]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden[-1])   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs, dim=-1)
            
        return output

In [33]:
# Define hyperparameters
size_of_vocab = len(Review.vocab)
embedding_dim = 300
num_hidden_nodes = 200
num_output_nodes = 5
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [34]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(20826, 300)
  (encoder): LSTM(300, 200, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=200, out_features=5, bias=True)
)
The model has 6,972,005 trainable parameters


In [35]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [36]:
def train_loop(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        review, review_lengths = batch.review   
        
        # convert to 1D tensor
        predictions = model(review, review_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels.long())
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [37]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            review, review_lengths = batch.review
            
            # convert to 1d tensor
            predictions = model(review, review_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels.long())
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [38]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train_loop(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.410 | Train Acc: 49.85%
	 Val. Loss: 1.408 |  Val. Acc: 49.68% 

	Train Loss: 1.405 | Train Acc: 49.97%
	 Val. Loss: 1.408 |  Val. Acc: 49.68% 

	Train Loss: 1.405 | Train Acc: 49.97%
	 Val. Loss: 1.408 |  Val. Acc: 49.67% 

	Train Loss: 1.405 | Train Acc: 49.97%
	 Val. Loss: 1.408 |  Val. Acc: 49.68% 

	Train Loss: 1.405 | Train Acc: 49.99%
	 Val. Loss: 1.408 |  Val. Acc: 49.71% 

	Train Loss: 1.404 | Train Acc: 50.12%
	 Val. Loss: 1.405 |  Val. Acc: 50.02% 

	Train Loss: 1.400 | Train Acc: 50.40%
	 Val. Loss: 1.400 |  Val. Acc: 50.39% 

	Train Loss: 1.394 | Train Acc: 50.90%
	 Val. Loss: 1.392 |  Val. Acc: 51.09% 

	Train Loss: 1.387 | Train Acc: 51.48%
	 Val. Loss: 1.382 |  Val. Acc: 51.76% 

	Train Loss: 1.366 | Train Acc: 52.87%
	 Val. Loss: 1.363 |  Val. Acc: 52.94% 



In [39]:
path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

In [40]:
import spacy
nlp = spacy.load('en')

def classify_review(tweet):
    
    categories = {
        0: "Very Negative",
        1: "Negative",
        2: "Neutral",
        3: "Positive",
        4: "Very Positive"
      }
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[Label.vocab.stoi[pred.item()]]

In [43]:
classify_review("Weather is pleasant.")

'Neutral'

In [44]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

In [45]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

In [46]:
random_deletion('Farmers protest at New Delhi'.split(' '))


['protest', 'at', 'Delhi']

In [48]:
!pip install google_trans_new

In [49]:
import random
import google_trans_new

from google_trans_new import google_translator

translator = google_translator()
sentence = ['The dog slept on the rug', 'This is good coffee']

available_langs = list(translator.LANGUAGES.keys()) 
trans_lang = random.choice(available_langs) 
print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

translations = translator.translate(sentence, lang_tgt=trans_lang)
print(translations)
# t_text = [t for t in translations]
# print(t_text)

translations_en_random = translator.translate(translations, lang_src=trans_lang, lang_tgt='en') 
print(translations_en_random)
# en_text = [t for t in translations_en_random]
# print(en_text)